In [1]:
import os
from tqdm import tqdm
from mmseg.apis import inference_model, init_model
import numpy as np
import shutil
import matplotlib.pyplot as plt

In [35]:
import os
import shutil
import random

# Set paths for your dataset
base_dir = "mmsegmentation/data/loveDA"
img_dir = os.path.join(base_dir, "img_dir/train")
ann_dir = os.path.join(base_dir, "ann_dir/train")

# Output paths
train_initial_img_dir = os.path.join(base_dir, "img_dir/train_initial")
train_pool_img_dir = os.path.join(base_dir, "img_dir/train_pool")
train_initial_ann_dir = os.path.join(base_dir, "ann_dir/train_initial")

# Create directories if they don't exist
os.makedirs(train_initial_img_dir, exist_ok=True)
os.makedirs(train_pool_img_dir, exist_ok=True)
os.makedirs(train_initial_ann_dir, exist_ok=True)

# Parameters
initial_subset_size = 50  # Number of images to include in the initial labeled set

# List all image files in the training directory
all_images = [f for f in os.listdir(img_dir) if f.endswith('.png') or f.endswith('.jpg')]

# Shuffle the dataset
random.shuffle(all_images)

# Split into initial labeled set and unlabeled pool
initial_images = all_images[:initial_subset_size]
pool_images = all_images[initial_subset_size:]

# Copy files to their respective directories
for img_file in initial_images:
    # Copy image to train_initial
    shutil.copy(os.path.join(img_dir, img_file), os.path.join(train_initial_img_dir, img_file))

    # Copy corresponding annotation to train_initial
    ann_file = img_file.replace('.jpg', '.png').replace('.png', '_label.png')  # Adjust extension if needed
    ann_path = os.path.join(ann_dir, ann_file)
    if os.path.exists(ann_path):
        shutil.copy(ann_path, os.path.join(train_initial_ann_dir, ann_file))

for img_file in pool_images:
    # Copy image to train_pool
    shutil.copy(os.path.join(img_dir, img_file), os.path.join(train_pool_img_dir, img_file))

print("Dataset split completed.")

Dataset split completed.


In [38]:
config_file = 'mmsegmentation/configs/deeplabv3plus/deeplabv3plus_r18-d8_4xb4-80k_loveda-512x512_active.py'
checkpoint_file = 'mmsegmentation/work_dirs/deeplabv3plus_r18-d8_4xb4-80k_loveda-512x512/iter_8000_active.pth'  # Adjust if needed
model = init_model(config_file, checkpoint_file, device='cuda:0')

Loads checkpoint by local backend from path: mmsegmentation/work_dirs/deeplabv3plus_r18-d8_4xb4-80k_loveda-512x512/iter_16000.pth


In [39]:
unlabeled_dir = 'mmsegmentation/data/loveDA/img_dir/train_pool'
unlabeled_images = [os.path.join(unlabeled_dir, f) for f in os.listdir(unlabeled_dir) if f.endswith('.png')]

In [50]:
import torch


uncertainties = []

for img_path in tqdm(unlabeled_images):
    # Perform inference
    result = inference_model(model, img_path)
    
    # Extract the logits from the `SegDataSample` object
    logits = result.seg_logits.data  # Access the logits tensor
    
    # Convert logits to probabilities
    probs = torch.softmax(logits, dim=0).cpu().numpy()
    
    # Compute pixel-wise entropy and average over the image
    entropy = -np.sum(probs * np.log(probs + 1e-10), axis=0).mean()  # Compute average entropy
    uncertainties.append((img_path, entropy))

# Sort images by uncertainty (descending order)
uncertainties.sort(key=lambda x: x[1], reverse=True)

100%|██████████| 2472/2472 [12:27<00:00,  3.31it/s]


In [52]:
#save the uncertainties

with open('uncertainties.txt', 'w') as f:
    for item in uncertainties:
        f.write("%s\n" % str(item))

In [62]:
img_dir_pool = 'mmsegmentation/data/loveDA/img_dir/train_pool'
ann_dir_pool = 'mmsegmentation/data/loveDA/ann_dir/train'
img_dir_initial = 'mmsegmentation/data/loveDA/img_dir/train_initial'
ann_dir_initial = 'mmsegmentation/data/loveDA/ann_dir/train_initial'

In [63]:
selected_images = [x[0] for x in uncertainties[:40]]
for img_path in selected_images:
    img_name = os.path.basename(img_path)
    ann_name = img_name  # Adjust as needed
    
    # Move image
    # shutil.move(os.path.join(img_dir_pool, img_name), os.path.join(img_dir_initial, img_name))
    
    # Move annotation
    shutil.move(os.path.join(ann_dir_pool, ann_name), os.path.join(ann_dir_initial, ann_name))
    # ann_path = os.path.join(ann_dir_pool, ann_name)
    # if os.path.exists(ann_path):
    #     shutil.move(ann_path, os.path.join(ann_dir_initial, ann_name))

print(f"Moved {len(selected_images)} images and annotations to the labeled set.")

FileNotFoundError: [Errno 2] No such file or directory: 'mmsegmentation/data/loveDA/ann_dir/train\\1931.png'

In [ ]:
# %cd mmsegmentation
!python tools/train.py configs/deeplabv3plus/deeplabv3plus_r18-d8_4xb4-80k_loveda-512x512_active.py --work-dir work_dirs/deeplabv3plus_r18-d8_4xb4-80k_loveda-512x512_active --resume -


12/12 12:03:14 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 295596494
    GPU 0: NVIDIA GeForce GTX 1650 Ti
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6
    NVCC: Cuda compilation tools, release 12.6, V12.6.85
    MSVC: Microsoft (R) C/C++ Optimizing Compiler Version 19.29.30143 for x64
    GCC: n/a
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 193431937
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtim

c:\users\suyog\desktop\monsoon_24\cv\computer-vision-project\mmsegmentation\mmseg\models\backbones\resnet.py:431: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '
c:\users\suyog\desktop\monsoon_24\cv\computer-vision-project\mmsegmentation\mmseg\models\losses\cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
c:\users\suyog\desktop\monsoon_24\cv\computer-vision-project\mmsegmentation\mmseg\engine\hooks\visualization_hook.py:60: UserWarning: The draw is False, it means that the hook for visualization will not take effect. The results will NOT be visualized or stored.
  warnings.warn('The draw is False, it means that the '
c:\Users\suyog\Desktop\monsoon_24\CV\computer-vis